In [1]:
import pandas as pd
import re, hashlib, math, time
from random import randint, seed
seed(1631996)

Вбила в поиск "алгоритмы поиска дубликатов в текстовых данных"

Нашла статью https://habr.com/ru/companies/mts_ai/articles/726012/

пошла искать MinHash

нашла код https://www.codemotion.com/magazine/backend/fast-document-similarity-in-python-minhashlsh/

адоптировала под данные https://www.kaggle.com/datasets/mikhailma/russian-social-media-text-classification

которые предварительно почистила : удалила все ссылки, убрала цифры, удалила стоп-слова, провела лемматизацию. Удалила явные дубликаты

в код сильно не вникала

In [2]:
class hashFamily:
    def __init__(self, i):
        self.resultSize = 8 # how many bytes we want back
        self.maxLen = 20 # how long can our i be (in decimal)
        self.salt = str(i).zfill(self.maxLen)[-self.maxLen:]
        
    def get_hash_value(self, el_to_hash):
        return int(hashlib.sha1(str(el_to_hash).encode('utf-8') + self.salt.encode('utf-8')).hexdigest()[-self.resultSize:], 16)
    

In [3]:
class shingler:
    def __init__(self, k):
        
        if k > 0:
            self.k = int(k)
        else:
            self.k = 10
        
    #inner class utility
    def process_doc(self, document):
        return re.sub("( )+|(\n)+"," ",document).lower()
    
    def get_shingles(self, document):
        shingles = set()
        document= self.process_doc(document)
        for i in range(0, len(document)-self.k+1 ):
            shingles.add(document[i:i+self.k])
        return shingles
    
    def get_k(self):
        return self.k
    
    #return sorted hash
    def get_hashed_shingles(self, shingles_set):
        hash_function = hashFamily(0)
        return sorted( {hash_function.get_hash_value(s) for s in shingles_set} )
        

In [4]:
class minhashSigner:
    def __init__(self, sig_size):
        self.sig_size=sig_size
        self.hash_functions = [hashFamily(randint(0,10000000000)) for i in range(0,sig_size)]
    
    def compute_set_signature(self, set_):
        set_sig = []
        for h_funct in self.hash_functions:
            min_hash = math.inf
            for el in set_:
                h = h_funct.get_hash_value(el)
                if h < min_hash:
                    min_hash = h
                
            set_sig.append(min_hash)
        
        return set_sig
    
    #return a list of lists that can be seen as the signature matrix
    def compute_signature_matrix(self, set_list):
        signatures = []
        for s in set_list:
            signatures.append( self.compute_set_signature(s) )
            
        return signatures

In [5]:
class lsh:
    def __init__(self, threshold=0.8):
        self.threshold = threshold
        
    def get_signature_matrix_bands(self, sig_matrix, bands_nr, sign_len): 
        #bands_nr = b
        #sign_len = n
        r = int(sign_len/bands_nr) #number of rows in each band
        bands = {} # {band_nr: [col_1,col_2,...]} where col_1 is all the values of Sig(S_i) for band b.
        for i in range(0,bands_nr):
            bands[i] = []
        
        # put Subsets of the columns of signature matrix into the appropriate bucket and cosider a column 
        # as a unique block so that we can hash the entire column.
        # Basically a band is a list of element, where each element is a subset of a signature of a given set.
        for signature in sig_matrix: 
            
            for i in range(0, bands_nr):
                idx = i*r    
                bands[i].append(' '.join(str(x) for x in signature[idx:idx+r]) ) 
                    
        return bands

    #band is a list 
    # construct a dictionary {hash(band_column): doc_id that produced this hash}
    def get_band_buckets(self, band, hash_funct):
        buckets = {}
        for doc_id in range(0,len(band)):
            value = hash_funct.get_hash_value( band[doc_id] )
            if value not in buckets:
                buckets[value] = [doc_id]
            else:
                 buckets[value].append(doc_id)
                
        return buckets
    
    def get_candidates_list(self, buckets):
        candidates = set()
        # buckets is a dictionary containing key=bucket, value= list of doc_ids that hashed to bucket
        for bucket,candidate_list in buckets.items():
            if len(candidate_list) > 1:
                for i in range(0,len(candidate_list)-1):
                    for j in range(i+1,len(candidate_list)):  
                        pair = tuple(sorted( (candidate_list[i],candidate_list[j]) ))
                        candidates.add(pair)
                
        return candidates #ie a set of couples, each couple is a candidate pair
    
    def check_candidates(self, candidates_list, threshold, sigs):
        similar_docs = set() #set of tuples
        # similar_pair is a couple containing doc_ids of documents that hashed to same bucket
        for  similar_pair in candidates_list:
            #for all the pairs of document in the list check similarity of their signatures
            doc_id_1 = similar_pair[0]
            doc_id_2 = similar_pair[1]
            signature_1 = set(sigs[doc_id_1]) #get the i-th column from signature matrix where i is doc_id in the collision list
            signature_2 = set(sigs[doc_id_2])
            js = len(signature_1.intersection(signature_2)) /len(signature_1.union(signature_2))
            
            if js >= threshold:
                similar_docs.add( tuple(sorted((doc_id_1,doc_id_2) )) )
                        
                        
        return similar_docs
    
    def get_similar_items(self, sig_matrix, bands_nr, sign_len):
        similar_docs = set()
        #divide signature matrix into bands
        bands = lsh_instance.get_signature_matrix_bands(sig_matrix,bands_nr,sign_len)
        
        #for all the bands
        for band_id, elements in bands.items():
            #produce the buckets for the given band (band_id) with a random hash function
            buckets = lsh_instance.get_band_buckets(elements, hash_funct=hashFamily(randint(0,10000000000)))
            #Get all the candidate pairs
            candidates = lsh_instance.get_candidates_list(buckets)
            #Check all candidate pairs' signatures
            for sim_tuple in lsh_instance.check_candidates(candidates, self.threshold, sig_matrix):
                similar_docs.add( sim_tuple)

        return similar_docs #return all the similar signatures that respect the threshold
                
        

In [6]:
dataset=pd.read_csv("Data/x_train.csv", index_col=0)
dataset.head()

,lemmas_string
12073,международный федерация шахматы фиде объявить ...
19379,приветствовать ставить штатный пружина сцеплен...
31389,друг обратить внимание сделать футболка символ...
26252,рождение тахир свой рождение отмечать нападать...
25310,внимание игрок гость ярославль комуса фест дат...


In [7]:
print("Loading dataset...")
dataset=pd.read_csv("Data/x_train.csv", index_col=0)
dataset['doc_id']=dataset.index
doc_nr = dataset['doc_id'].max()
print("Dataset loaded correctly.")
print("Producing Shingles...")
start_time = time.time()
#an array where the index i represent the document_id and the element shingling_list[i] the hashed shingles for document document_id
shingling_list = [None] * (doc_nr +1) 
shingling_size = 10
signature_size = 50
bands_nr = 10

shingler_inst = shingler(shingling_size)
signer = minhashSigner(signature_size)


#produce hashed shinglings for all documents
for index, row in dataset.iterrows():
    doc = row['lemmas_string']
    i = row['doc_id']
    
    shinglings = shingler_inst.get_hashed_shingles( shingler_inst.get_shingles(doc) )
    shingling_list[i] = shinglings

end_time = time.time()
lt1 = end_time - start_time
print("Shingles produced in:\t %.2f seconds."%(lt1))


start_time = time.time()
print("Computing signature matrix...")
#produce a signature for each shingle set
signature_matrix = signer.compute_signature_matrix( shingling_list )
end_time = time.time()
lt2 = end_time - start_time
print("Signature Matrix computed in:\t %.2f seconds." %(lt2))

Loading dataset...
Dataset loaded correctly.
Producing Shingles...
Shingles produced in:	 25.52 seconds.
Computing signature matrix...
Signature Matrix computed in:	 918.09 seconds.


In [8]:
lsh_instance = lsh(threshold=0.8)
start_time = time.time()
print("Computing LSH similarity...")
lsh_similar_itemset = lsh_instance.get_similar_items(signature_matrix, bands_nr, signature_size)
end_time = time.time()
lsh_computation_time = end_time - start_time
print("LSH Similarity computed in:\t %.2f seconds.\nSimilar Elements Found: %d" %(lsh_computation_time,len(lsh_similar_itemset)))


Computing LSH similarity...
LSH Similarity computed in:	 2.06 seconds.
Similar Elements Found: 300


In [9]:
print(lsh_computation_time)
print(len(lsh_similar_itemset))

2.064232587814331
300


In [10]:
dataset['list_similar'] = 0

In [11]:
num_similar = 1
for i in lsh_similar_itemset:
    if dataset.at[i[0], 'list_similar'] == 0 and dataset.at[i[1], 'list_similar'] == 0:
        dataset.at[i[0], 'list_similar'] = num_similar
        dataset.at[i[1], 'list_similar'] = num_similar
        num_similar +=1
      
    elif dataset.at[i[0], 'list_similar'] == 0 and dataset.at[i[1], 'list_similar'] != 0:
        num_old = dataset.at[i[1], 'list_similar']
        dataset.at[i[0], 'list_similar'] = num_old
    elif dataset.at[i[0], 'list_similar'] != 0 and dataset.at[i[1], 'list_similar'] == 0:
        num_old = dataset.at[i[0], 'list_similar']
        dataset.at[i[1], 'list_similar'] = num_old
    elif dataset.at[i[0], 'list_similar'] < dataset.at[i[1], 'list_similar']:
        num_old = dataset.at[i[0], 'list_similar']
        dataset.at[i[1], 'list_similar'] = num_old
    elif dataset.at[i[0], 'list_similar'] > dataset.at[i[1], 'list_similar']:
        num_old = dataset.at[i[1], 'list_similar']
        dataset.at[i[0], 'list_similar'] = num_old

In [12]:
dataset.query('list_similar==29')

,Unnamed: 0,lemmas_string,doc_id,list_similar
796,37290,остаться место успеть внимание болельщик толья...,796,29
6595,7198,внимание болельщик тольятти организоваться пое...,6595,29


In [13]:
print(f'данный алгоритм нашел еще {dataset.list_similar.nunique()} дупликатов за {lt1+lt2+lsh_computation_time}')

данный алгоритм нашел еще 216 дупликатов за 2.064232587814331
